In [ ]:
from importlib import reload
import os
import time

import lc_analysis_function
reload (lc_analysis_function)
from lc_analysis_function import *
route= os.getcwd()

#warnings check directories;
for fold in ['Steady_States','Lightcurve_Simulation','Results']:
    if fold not in os.listdir(): 
        print('Sorry, subfolder {} is necessary for this routine.\n Please follow README.txt instructions'.format(fold))
        exit()

### RUN FOR A SINGLE LIGHTCURVE AND CONTINUE IF THE CODE STUCK 
BY ALLOWING DISCONTINUITY  & STITCHING TOGETHER SEGMENTS

In [ ]:
#OPTIONS
RESTART= True #### 1 to activate / Must use for 'delta'
STEADY=  False ##### 1: new steady state
obj='Mrk421' #no '-' or '_' or '/', raw letters and numbers
name='56397'
lendays=121 #753 ###Desired time interval (days) for reproducing lightcurves 
newDELC='no' #'on'/ 'no' to create a new artificial timecurve with DELgen code,
             #set to 'no' if input LC is set and you want to change parameter and/or power-law transformation
namevar='B' ##Quantity to be varying  'le', 'B', 'lext' /  'delta use different script
POWER=1.5 ####power p where: param propto (Flux)^p / 1.0 for simple use 		   #example:  for delta POWER= '0.25'

#MK code set up
TOLinit=-5 ###TOL= 1e"TOLinit" : logarithm of steady state tolerance used in var.f too
noise_e=-37.0
RESOLUTION_electrons = 5 # cut the energy decade into that many segments e.g. 5 > ΔlogE = 1.0/5 = 0.2
#settings of the integration loops
TOLlow ,TOLhigh = TOLinit , -3 #integers, scan between these values when trying to surpass a difficult point
runnum_max = 30

In [4]:
def compile_codevar(ifake,namevar,route, name='code'):
    os.system('sed -i -e "205s/.*/{}/" code_var_{}.f'.format('       ifake={}'.format(ifake),namevar))
    os.system('g77-3.4 -o code_var code_var_{}.f -lnag -O3'.format(namevar)) #-funroll-loops
    os.system('cp {}/Steady_States/{}.inp {}/code.inp'.format(route,name,route))

def calc_mod(nowtime,STEPS,TIME,TOLs,nplus, LOG_NOISE_E=noise_e, RESdex= RESOLUTION_electrons):
        write_input_file('code',STEPout=STEPS, TIME=TIME, TOL=TOLs , LOG_NOISE_E=noise_e ,RESdex= RESOLUTION_electrons)
        DT=TIME/STEPS
        rshT=(TIME-nowtime)/DT
        with open('dum_r4.dat','r') as dum:
            first = dum.readline()
            oldtime = float(first.split()[1])
            rest= dum.readlines()
        new_dum = [str(rshT)+' '+str(oldtime)+'\n']+rest
        newdum=open('dum_r4.dat','w')
        newdum.writelines(["%s" % item.replace('\n','')  for item in new_dum])
        newdum.close()
        mod=nowtime//TIME
        return mod, oldtime

def calc_modstep(nowtime,DT,TOLs,nplus, LOG_NOISE_E=noise_e, RESdex= RESOLUTION_electrons):
        shT= round(nowtime+nplus*DT,1)
        shstp=shT/DT
        write_input_file('code',START=1,TOL=TOLs, TIME=shT, STEPout=shstp,  LOG_NOISE_E=noise_e, RESdex= RESOLUTION_electrons)
        rshT=(shT -nowtime)/DT
        with open('dum_r4.dat','r') as dum:
            first = dum.readline()
            oldtime = float(first.split()[1])
            rest= dum.readlines()
        if rshT!=0.0:
            new_dum = [str(rshT)+' '+str(oldtime)+'\n']+rest
            newdum=open('dum_r4.dat','w')
            newdum.writelines(["%s" % item.replace('\n','')  for item in new_dum])
            newdum.close()
        os.system('{}/code_var'.format(route))
        with open('dum_r4.dat','r') as dum:
            first = dum.readline()
            nowtime = float(first.split()[1])
        modstep= (nowtime-oldtime)//DT
        return modstep,nowtime, oldtime

In [ ]:
#SCRIPT
#constants
fktc='fakeTC_{}_{}_{}.txt'.format(namevar,obj,name)
keep_outputs= [81, 85, 89,55]+(namevar=='lext')*[86] #which fortran files to save
route_save = '{}/Results/{}_^{}'.format(route,namevar,POWER)
#route_save = '{}/Results/{}/{}_^{}'.format(route,obj,namevar,POWER) #if doing many object

#Steady State (uncomment if changed object), runs locally
if STEADY:
    print("Steady state t=5.0tcross being made")
    os.system("rm steady*")
    os.system("cp ./code.inp ./temp.inp")
    os.system("cp {}/Steady_States/{}.inp ./code.inp".format(route,name))
    os.system("./code_noprint")
    os.system("cp ./temp.inp ./code.inp")
    os.system("rm temp.inp")
    os.system("cp ./fort.81 ./steady.81")
    os.system("cp ./fort.85 ./steady.85")
    os.system("cp ./fort.89 ./steady.89")
    print("Steady State ready/ Var Code next")

#used variables in script
file = open(route+'/Steady_States/{}_new4.inp'.format(name),'r')
R, delta = 0.0 , 0.0
for i,line in enumerate(file.readlines()):
              if i==2: R =float(line.split()[0])
              if i==7: delta = float(line.split()[0])
if R==0.0 or delta==0.0: 
    print('ERROR IN READING STEADY STATE INPUT FILE, TERMINATING')
    exit()
tcross=R/3.0e10/24.0/3600.0
              

if RESTART:
    #Run code from where it has crashed or from the beginning
    os.system('rm fort* fakeTC*')
    #Create/Display_current Timecurve
    remT=lendays/tcross*delta
    #remT+=1
    runnum=0
    oldtime=0.0
else:
    #Continue using log
    runlog=np.loadtxt('./Lightcurve_Simulation/multi.log')
    runnum = runlog[-1] -1
    print("WARNING: Restart at the run number"+str(runnum))
    print("For different treatment (not overwriting the crashed run) use run_var.sh")
    lenprev=runnum+1
    remT= lendays/tcross*delta-lenprev
    remT+=1 #extended beyond requested time to guarantee with the roundings the output will be correct in len
#loop that produces discontinued segments when further integration of the code is not possible
#gaps maybe introduced
crashes, stuck,  tolchange =0 , 0  , 0 
SECONDS = time.time()
while remT>0:
    os.system("rm fort.* fakeTC.txt")
    newD='multi'+str(remT)
    os.system("cp {}/Steady_States/{}.inp {}/Lightcurve_Simulation/code.inp".format(route,name,route))
    os.system("cp {}/Steady_States/{}_new4.inp {}/Lightcurve_Simulation/fkTC.inp".format(route,name,route))
    TIME, STEPS, ifake = lc_analysis(route, lendays, newD, POWER, namevar, name,ALLOW_MULTI=False, 
                SAVE_LOG = False, PLOT=False, obj=obj,DATA_PROCESS=False)
    DT = TIME/STEPS
    #write_input_file('code',route='./Steady_States',STEPout=STEPS, TIME=TIME)
    write_input_file(name,route='./Steady_States',STEPout=STEPS, TIME=TIME,LOG_NOISE_E=noise_e,RESdex= RESOLUTION_electrons)
    write_input_file(name+'_new4',route='./Steady_States',STEPout=STEPS, TIME=TIME,LOG_NOISE_E=noise_e,RESdex= RESOLUTION_electrons)
    if namevar == "gmax": os.system('sed -i -e "1s/5/10/1" code.inp') #change energy binning for this mode
    if runnum>0:     
        print("\n New Run {} initialized".format(runnum))
        print("Check Steepness Diagram in Lightcurve folder to find potential NAG-failure points")            
    print("Time interval for the run created {} tcross".format(TIME))
    
    #write and compile fortran radiative code with the new input file to be read
    compile_codevar(ifake,namevar,route, name)
    os.system('{}/code_var'.format(route))
    
    TOL = TOLinit
    TOLs ='1e'+str(TOLinit)
    nowtime, mod = 0.0, 0
    while mod==0:
        modstep , nowtime, oldtime = calc_modstep(nowtime,DT,TOLs,0.0, LOG_NOISE_E=noise_e, RESdex= RESOLUTION_electrons)
        if modstep==0.0:
            TOL=TOLhigh
            tolchange+=1 #counting TOL changes not sub-changes in the first digit precision
            nplus=10.
            while modstep==0:
                for subTOL in [2,3]: #twitching the tolerance sometimes helps the integrator
                    TOLs=str(subTOL)+'e'+str(int(TOL))
                    #runs the code and outputs the interval remaining
                    modstep, nowtime, oldtime = calc_modstep(nowtime,DT,TOLs,nplus,
                                LOG_NOISE_E=noise_e, RESdex= RESOLUTION_electrons)
                    if modstep==1.0: 
                        break
                    else:
                        crashes+=1
                TOL+= -1
                tolchange+=1 #counting TOL changes not sub-changes in the first digit precision
                if TOL==TOLlow: 
                    nplus=1. #reduce added time for lower tolerence (more precise calculation)
                elif TOL < TOLlow:
                    modstep=1.0
                    break
            #end modstep while
        # if modstep succeeded or not (cannot tell by modstep==1.0)
        if TOL >= TOLlow:
            #restore initial conditions, before any  chenge in tolerence was induced
            TOL=TOLinit
            TOLs="1e"+str(TOL)
            mod, oldtime = calc_mod(nowtime,STEPS,TIME,TOLs,nplus, LOG_NOISE_E=noise_e, RESdex= RESOLUTION_electrons)
            if mod==0: crashes+=1
        else:
            print("Code STUCK at time tcross".format(nowtime))
            mod=1
    #end mod while
    for numt in keep_outputs:
        os.system("cp fort.{} fort_{}.{}".format(str(numt),name,str(numt)))
    runnum +=1
    remT=TIME-nowtime
    print('Simulation number created to conjoin : '+str(runnum))
    if float(runnum)>runnum_max: 
        print('I am looping here. I will stop and do not generate more segments.\n
                If these was indeed a long time-curve input just continue runnum_max greater\n and runnum =0 => previous value ')
        remT ==0.0 #do not allow infinite interrupts due to numerical errors
    stuck+=1
#outer loop end (while remT>0)
end=time.time()
SECONDS = end-SECONDS

if remT<2.0:
    print( "Code run to the endtime ")
    print("Code crashed {} times due to NAG FAILURE".format(crashes))
    print("Code stucked {} times due to NAG FAILURE".format(stuck))
    print("Last time tolerance changed {} times".format(tolchange))
    print("Total elapsed time: {} min".format(round(SECONDS/60.0,1)))
    os.system('mkdir {}'.format(route_save))
    #os.sytem('mkdir {}/Results/{}/{}_^{}".format(route,obj,namevar,POWER)
    for ff in ['fort*', 'steady*', 'code.inp' ,'fakeTC*', 'Var*', 'code_var_'+namevar+'.f', 'code_var']:
          os.system('cp {}/{} {}/'.format(route,ff,route_save))
    os.system('rm temp.inp fort.* fakeTC*') #clean 
    print("Results ready in directory {}".format(route_save))
else:
    print('Abnormal exit, code maybe interrupted \n OR \n LENDAYS too short and did not run')

In [ ]:
#run python script into the Results folder from here 
#// Analysis (Var4.py) next , Ctrl+Z to stop here"
os.system('python3 {}/Var7.py {}/'.format(route_save,route_save)) #run_analysis on exit

# RUN FOR DIFFERENT SEGMENTS (APPLICATION MRK421)

In [ ]:
#OPTIONS
import numpy as np
obj='Mrk421' #no '-' or '_' or '/', raw letters and numbers
newDELC='no' #'on'/ 'no' to create a new artificial timecurve with DELgen code,
             #set to 'no' if input LC is set and you want to change parameter and/or power-law transformation
namevar='B' ##Quantity to be varying  'le', 'B', 'lext' /  'delta use different script
POWER=1.5 ####power p where: param propto (Flux)^p / 1.0 for simple use #example:  for delta POWER= '0.25'

#double parameter variations with the second one based on the model -observations residuals of the first one
DOUBLE_VAR  = True  #if false only the initial_var will run
initial_var = 'p' #overwrites namevar above, but uses POWER (if applicable)
segments = [] #['56393half']+list(np.linspace(56395,56400,6).astype(int))  #initial running on these many segments

#data to compare model - observations maybe shorter, different definition of segments for second parameter
RESIDUALS_BAND ='MAGIC'
follow_up_var='le'
POWER_B = 0.75
segments_B = list(np.linspace(56393,56400,8).astype(int)) #index of segments (it can be days if it's is relevant to your study, it's just an index)

#MK code set up
TOLinit=-5 ###TOL= 1e"TOLinit" : logarithm of steady state tolerance used in var.f too
noise_e=-37.0
RESOLUTION_electrons = 5 # cut the energy decade into that many segments e.g. 5 > ΔlogE = 1.0/5 = 0.2
#settings of the integration loops
TOLlow ,TOLhigh = TOLinit , -3 #integers, scan between these values when trying to surpass a difficult point
runnum_max = 30

In [12]:
def compile_codevar(ifake,namevar,route, name='code'):
    os.system('sed -i -e "205s/.*/{}/" code_var_{}.f'.format('       ifake={}'.format(ifake),namevar))
    os.system('g77-3.4 -o code_var code_var_{}.f -lnag -O3'.format(namevar)) #-funroll-loops
    os.system('cp {}/Steady_States/{}.inp {}/code.inp'.format(route,name,route))

def calc_mod(nowtime,STEPS,TIME,TOLs,nplus, LOG_NOISE_E=noise_e, RESdex= RESOLUTION_electrons):
        write_input_file('code',STEPout=STEPS, TIME=TIME, TOL=TOLs , LOG_NOISE_E=noise_e ,RESdex= RESOLUTION_electrons)
        DT=TIME/STEPS
        rshT=(TIME-nowtime)/DT
        with open('dum_r4.dat','r') as dum:
            first = dum.readline()
            oldtime = float(first.split()[1])
            rest= dum.readlines()
        new_dum = [str(rshT)+' '+str(oldtime)+'\n']+rest
        newdum=open('dum_r4.dat','w')
        newdum.writelines(["%s" % item.replace('\n','')  for item in new_dum])
        newdum.close()
        mod=nowtime//TIME
        return mod, oldtime

def calc_modstep(nowtime,DT,TOLs,nplus, LOG_NOISE_E=noise_e, RESdex= RESOLUTION_electrons):
        shT= round(nowtime+nplus*DT,1)
        shstp=shT/DT
        write_input_file('code',START=1,TOL=TOLs, TIME=shT, STEPout=shstp,  LOG_NOISE_E=noise_e, RESdex= RESOLUTION_electrons)
        rshT=(shT -nowtime)/DT
        with open('dum_r4.dat','r') as dum:
            first = dum.readline()
            oldtime = float(first.split()[1])
            rest= dum.readlines()
        if rshT!=0.0:
            new_dum = [str(rshT)+' '+str(oldtime)+'\n']+rest
            newdum=open('dum_r4.dat','w')
            newdum.writelines(["%s" % item.replace('\n','')  for item in new_dum])
            newdum.close()
        os.system('{}/code_var'.format(route))
        with open('dum_r4.dat','r') as dum:
            first = dum.readline()
            nowtime = float(first.split()[1])
        modstep= (nowtime-oldtime)//DT
        return modstep,nowtime, oldtime

In [ ]:
#route_save = '{}/Results/{}/{}_^{}'.format(route,obj,namevar,POWER) #if doing many object
from importlib import reload
import os
import time

import lc_analysis_function
reload (lc_analysis_function)
from lc_analysis_function import *
route= os.getcwd()

#warnings check directories;
for fold in ['Steady_States','Lightcurve_Simulation','Results']:
    if fold not in os.listdir(): 
        print('Sorry, subfolder {} is necessary for this routine.\n Please follow README.txt instructions'.format(fold))
        exit()
        
if DOUBLE_VAR:
    params_to_run_in_that_order  = [initial_var, follow_up_var]
    if not (initial_var=='p' and follow_up_var=='le') or\
    (initial_var=='lext' and follow_up_var=='le'):
            print('Sorry combination of parameters not implemented.\n Also check order')
            exit()
else:
    params_to_run_in_that_order = [namevar]
    
#used variables in script
SECONDS = time.time()
namecodevar = initial_var
for inv, namevar in enumerate(params_to_run_in_that_order):
    print('Running parameter with variations '+namevar)
    datfile_follow_up = None
    if namevar == follow_up_var: 
        datfile_follow_up = 'residuals_'+initial_var+'_'+RESIDUALS_BAND+'.txt'
        segments = segments_B
        POWER = POWER_B
        namecodevar += '+'+follow_up_var
    route_save = '{}/Results/{}_^{}'.format(route,namecodevar,POWER)
    keep_outputs= [81, 85, 89,55]+(namevar=='lext')*[86]
    for nameseg in segments:
        fktc='fakeTC_{}_{}_{}.txt'.format(namevar,obj,nameseg)
        file = open(route+'/Steady_States/{}_new4.inp'.format(nameseg),'r')
        R, delta = 0.0 , 0.0
        for i,line in enumerate(file.readlines()):
                      if i==2: R =float(line.split()[0])
                      if i==7: delta = float(line.split()[0])
        if R==0.0 or delta==0.0: 
            print('ERROR IN READING STEADY STATE INPUT FILE, TERMINATING')
            exit()
        tcross=R/3.0e10/24.0/3600.0

        #Run code from where it has crashed or from the beginning
        #Create/Display_current Timecurve
        runnum=nameseg
        oldtime=0.0
        TOL = TOLinit
        os.system("cp {}/Steady_States/{}.inp {}/Lightcurve_Simulation/code.inp".format(route,nameseg,route))
        os.system("cp {}/Steady_States/{}_new4.inp {}/Lightcurve_Simulation/fkTC.inp".format(route,nameseg,route))
        TIME, STEPS, ifake = lc_analysis(route, None , None, POWER, namevar, str(nameseg), ALLOW_MULTI=False, 
                    SAVE_LOG = False, PLOT=False, obj=obj,DATA_PROCESS=True, SEGMENTS=True,
                    DOUBLE_VAR=DOUBLE_VAR*(inv>0), datfile_follow_up=datfile_follow_up)
        DT = TIME/STEPS
        print('Variations of type {}(t) written in fakeTC.txt:'.format(namevar))
        #write_input_file('code',route='./Steady_States',STEPout=STEPS, TIME=TIME)
        write_input_file(str(nameseg),route='./Steady_States',STEPout=STEPS, TIME=TIME,LOG_NOISE_E=noise_e ,RESdex= RESOLUTION_electrons)
        write_input_file(str(nameseg)+'_new4',route='./Steady_States',STEPout=STEPS, TIME=TIME,LOG_NOISE_E=noise_e ,RESdex= RESOLUTION_electrons)
        if namevar == "gmax": os.system('sed -i -e "1s/5/10/1" code.inp') #change energy binning for this mode
        print("\n New Run for segment {} initialized".format(runnum))
        print("Time interval for the run created {} tcross".format(TIME))
        #write and compile fortran radiative code with the new input file to be read
        compile_codevar(ifake,namecodevar,route, nameseg)
        os.system('{}/code_var'.format(route))
            
        TOLs ='1e'+str(TOLinit)
        crashes, tolchange =0 , 0 
        nowtime = 0.0
        mod, oldtime = calc_mod(nowtime,STEPS,TIME,TOLs, 0.0, LOG_NOISE_E=noise_e, RESdex= RESOLUTION_electrons)
        while mod==0.0:
            modstep , nowtime, oldtime = calc_modstep(nowtime,DT,TOLs,0.0, LOG_NOISE_E=noise_e, RESdex= RESOLUTION_electrons)
            if modstep==0.0:
                TOL=TOLhigh
                tolchange+=1 #counting TOL changes not sub-changes in the first digit precision
                nplus=10.
                while modstep==0:
                    for subTOL in [2,3]: #twitching the tolerance sometimes helps the integrator
                        TOLs=str(subTOL)+'e'+str(int(TOL))
                        #runs the code and outputs the interval remaining
                        modstep, nowtime, oldtime = calc_modstep(nowtime,DT,TOLs,nplus,
                                    LOG_NOISE_E=noise_e, RESdex= RESOLUTION_electrons)
                        if modstep==1.0: 
                            break
                        else:
                            crashes+=1
                    TOL+= -1
                    tolchange+=1 #counting TOL changes not sub-changes in the first digit precision
                    if TOL==TOLlow: 
                        nplus=1. #reduce added time for lower tolerence (more precise calculation)
                    elif TOL < TOLlow:
                        modstep=1.0
                        break
                #end modstep while
            # if modstep succeeded or not (cannot tell by modstep==1.0)
            if TOL >= TOLlow:
                #restore initial conditions, before any  chenge in tolerence was induced
                TOL=TOLinit
                TOLs="1e"+str(TOL)
                mod, oldtime = calc_mod(nowtime,STEPS,TIME,TOLs,nplus, LOG_NOISE_E=noise_e, RESdex= RESOLUTION_electrons)
                if mod==0: crashes+=1
            else:
                print('CODE STUCK:This should not be happening for short times and legitimate variations')
                print('At segment '+str(nameseg))
                print("and after time {} tcross".format(nowtime))
                print('Terminating, please check \n\n 1) code.inp 2)fakeTC.txt 3)code_var_namevar')
                mod=1
                exit()
            #end mod while        os.system("cp fort.81 fort_{}.81".format(nameseg))
        for numt in keep_outputs:
            os.system("cp fort.{} fort_{}.{}".format(str(numt),nameseg,str(numt)))
        print('Simulation for period name: '+str(runnum))
        #outer loop on namesegs
        print('Code run to the endtime for ALL DAYS for parameter '+str(nameseg))
        os.system('python3 {}/Var7.py {}/'.format(route_save,route_save)) #run_analysis on exit
end=time.time()
SECONDS = end-SECONDS
print( "Code run to the endtime ")
print("Code crashed {} times due to NAG FAILURE".format(crashes))
print("Last time tolerance changed {} times".format(tolchange))
print("Total elapsed time: {} min".format(round(SECONDS/60.0,1)))
os.system('mkdir {}'.format(route_save))
#os.sytem('mkdir {}/Results/{}/{}_^{}".format(route,obj,namevar,POWER)
for ff in ['fort*', 'code.inp' ,'fakeTC*', 'Var*', 'code_var_'+namevar+'.f', 'code_var']:
      os.system('cp {}/{} {}/'.format(route,ff,route_save))
os.system('rm temp.inp fort.* fakeTC*') #clean 
print("Results ready in directory {}".format(route_save))

Running parameter with variations p
NEW_delc not provided. Set to False
NEW_delc not provided. Set to False
Read 227 lines of data

 Section 0 t in [56394.896345-56395.91557]
Added 1
Total points in this time inteval: 	16
Renorm factor 1.0

 Section 1 t in [56395.91557-56396.919460000005]
Added 1
Total points in this time inteval: 	31
Renorm factor 1.0

 Section 2 t in [56396.919460000005-56397.930555]
Added 1
Total points in this time inteval: 	34
Renorm factor 1.0

 Section 3 t in [56397.930555-56398.939205]
Added 1
Total points in this time inteval: 	22
Renorm factor 1.0

 Section 4 t in [56398.939205-56400.02562]
Added 1
Total points in this time inteval: 	8
Renorm factor 1.0

 Section 5 t in [56400.02562-56401.038929999995]
Added 1
Total points in this time inteval: 	8
Renorm factor 1.0

 Section 6 t in [56401.038929999995-56401.13806]
Added 1
Total points in this time inteval: 	8
Renorm factor 1.0
Extrapolated for gaps with steady state

 Segment initial segment t in [56392.88998

invalid integer: read unexpected character
apparent state: unit 13 named code.inp
last format: list io
lately reading direct formatted external IO
Aborted (core dumped)
invalid integer: read unexpected character
apparent state: unit 13 named code.inp
last format: list io
lately reading direct formatted external IO
Aborted (core dumped)
invalid integer: read unexpected character
apparent state: unit 13 named code.inp
last format: list io
lately reading direct formatted external IO
Aborted (core dumped)
invalid integer: read unexpected character
apparent state: unit 13 named code.inp
last format: list io
lately reading direct formatted external IO
Aborted (core dumped)
invalid integer: read unexpected character
apparent state: unit 13 named code.inp
last format: list io
lately reading direct formatted external IO
Aborted (core dumped)
invalid integer: read unexpected character
apparent state: unit 13 named code.inp
last format: list io
lately reading direct formatted external IO
Aborted 

CODE STUCK:This should not be happening for short times and legitimate variations
At segment 56393
and after time 0.0 tcross
Terminating, please check 

 1) code.inp 2)fakeTC.txt 3)code_var_namevar
Simulation for period name: 56393
Code run to the endtime for ALL DAYS for parameter 56393
NEW_delc not provided. Set to False
NEW_delc not provided. Set to False
Read 227 lines of data

 Section 0 t in [56394.896345-56395.91557]
Added 1
Total points in this time inteval: 	16
Renorm factor 1.0

 Section 1 t in [56395.91557-56396.919460000005]
Added 1
Total points in this time inteval: 	31
Renorm factor 1.0

 Section 2 t in [56396.919460000005-56397.930555]
Added 1
Total points in this time inteval: 	34
Renorm factor 1.0

 Section 3 t in [56397.930555-56398.939205]
Added 1
Total points in this time inteval: 	22
Renorm factor 1.0

 Section 4 t in [56398.939205-56400.02562]
Added 1
Total points in this time inteval: 	8
Renorm factor 1.0

 Section 5 t in [56400.02562-56401.038929999995]
Added 1


invalid integer: read unexpected character
apparent state: unit 13 named code.inp
last format: list io
lately reading direct formatted external IO
Aborted (core dumped)
invalid integer: read unexpected character
apparent state: unit 13 named code.inp
last format: list io
lately reading direct formatted external IO
Aborted (core dumped)
invalid integer: read unexpected character
apparent state: unit 13 named code.inp
last format: list io
lately reading direct formatted external IO
Aborted (core dumped)
invalid integer: read unexpected character
apparent state: unit 13 named code.inp
last format: list io
lately reading direct formatted external IO
Aborted (core dumped)
invalid integer: read unexpected character
apparent state: unit 13 named code.inp
last format: list io
lately reading direct formatted external IO
Aborted (core dumped)
invalid integer: read unexpected character
apparent state: unit 13 named code.inp
last format: list io
lately reading direct formatted external IO
Aborted 

CODE STUCK:This should not be happening for short times and legitimate variations
At segment 56394
and after time 0.0 tcross
Terminating, please check 

 1) code.inp 2)fakeTC.txt 3)code_var_namevar
Simulation for period name: 56394
Code run to the endtime for ALL DAYS for parameter 56394
NEW_delc not provided. Set to False
NEW_delc not provided. Set to False
Read 227 lines of data

 Section 0 t in [56394.896345-56395.91557]
Added 1
Total points in this time inteval: 	16
Renorm factor 1.0

 Section 1 t in [56395.91557-56396.919460000005]
Added 1
Total points in this time inteval: 	31
Renorm factor 1.0

 Section 2 t in [56396.919460000005-56397.930555]
Added 1
Total points in this time inteval: 	34
Renorm factor 1.0

 Section 3 t in [56397.930555-56398.939205]
Added 1
Total points in this time inteval: 	22
Renorm factor 1.0

 Section 4 t in [56398.939205-56400.02562]
Added 1
Total points in this time inteval: 	8
Renorm factor 1.0

 Section 5 t in [56400.02562-56401.038929999995]
Added 1


In [1]:
TOL

NameError: name 'TOL' is not defined

### Extra Non-thermal components (already pre-run in another script)
__SLOW -- QUASI-STATIONARY__
> The stationary (or slow) component rarely if at all varies within a certain segment.
    It can be selected to be time_dependent (run blavar with different parameters) 
    or a single state for each segment
    
__FAST -- EXTRA__
> The extra (fast) component is used to explain abrupt variability flares on a timescale a lot less than the 
    light crossing time of the main blob. Typically this state is much smaller and moves much faster delta>100.0

_FORMAT_ = _[ Use or not (True/False), days_with_the_state, name (to draw from Steady_States), Doppler factor, radius, route_of time-dependent run (optional: only if the component is varying with time with the MK radiative-code]

In [ ]:
route_S= route+"/Results/slowle_^0.5/"
route_E= route+"/Results/fastdelta_^0.25/"
nameflc_S='Fermi_DLgen.txt' # name of the file of the real LC for varying the slow state 
nameflc_E='None' # name of the file of the real LC for varying the extra state
days_with_lowstate , days_with_extrastate = segments,  [56395,56396,56397]
STATIONARY_chars = [True, days_with_lowstate, 'lowstate' ,  50.0, 4.9e15, route_S, nameflc_S]
EXTRA_chars =  [False, days_with_extrastate 'extrastate',STATIONARY_chars[2]*3.0, STATIONARY_chars[2]/2**5.0, route_E,nameflc_E]

In [ ]:
import Var_function
reload (Var_function)
from Var_function import *
#This is the main routine for plotting. However these are the general setting, 
#check in Var_analysis.py for the extended in-code setting 
# OR use(adjust) with your own plotting routines and run interactively directly in the route_save folder Var8.py
            #standard
Var_analysis(obj, route_save, ALLTOOLS=False , SAVE=True,  imtyp='.png', ELECTRONS=False ,#calc electr. properties
             SED=  'on',                      TIMELAPSE=False ,  #SED 'on'(lines)/'con'(contour)/'no'(skip)
             TIMECURVES=True,  CORR=    False ,TIMELAPSE_CORR=False, #plot timecurves,correlation plot(multiple frames)
             TIMELAGS=False, DCFcalc= False, SFcalc= False , STINGRAY=False, #time-lag analysis
             CALC_THEO_INDICES=True ,  # to study the effectiveness of your POWER selection
           #additional                           
            STATIONARY =  STATIONARY_chars,  EXTRA= EXTRA_chars,
            TIMEDEPENDENT_DATA_ON_PLOT = [True,route+'/data/seds_all_flares/'],
            feedback_VHE_on_extrastate =        False     )    